In [13]:
import numpy as np
import pandas as pd
import pyproj
import sqlite3
import xarray as xr
from tqdm import tqdm  # progress bar

class CASESTUDY1:
    
    def __init__(self, turbine_csv, db_path):
        """
        Initializes the CASESTUDY1 class.

        Args:
            turbine_csv (str): Path to the CSV file containing wind turbine data.
            db_path (str): Path to the SQLite database containing building data.

        Attributes:
            turbine_csv (str): Stores the path to the wind turbine CSV file.
            db_path (str): Stores the path to the SQLite database.
            transformer (pyproj.Transformer): A coordinate system transformer for converting coordinates.
            turbines_data (xarray.Dataset): Processed wind turbine data.
            buildings_data (xarray.Dataset): Processed building data.
        """
        self.turbine_csv = turbine_csv
        self.db_path = db_path
        self.transformer = self._init_transformer()
        self.turbines_data = self._load_and_process_turbines()
        self.buildings_data = self._load_and_process_buildings()

    def _init_transformer(self):
        """
        Initializes a coordinate system transformer to convert coordinates from the Swiss coordinate system (EPSG:2056)
        to the WGS84 coordinate system (EPSG:4326). This transformer will be used for transforming the coordinates
        of wind turbines and buildings in subsequent methods.

        Returns:
            pyproj.Transformer: Transformer object for converting coordinates between Swiss and WGS84 systems.
        """
        swiss = pyproj.CRS('EPSG:2056')
        wgs84 = pyproj.CRS('EPSG:4326')
        return pyproj.Transformer.from_crs(swiss, wgs84, always_xy=True)

    def _load_and_process_turbines(self):
        """
        Loads wind turbine data from a CSV file, filters operational turbines, and converts their coordinates
        from the Swiss coordinate system to the WGS84 system. This processed data is then structured into an xarray
        Dataset for convenient access and manipulation.

        The method specifically filters turbines that are in normal operational status, ensuring that the dataset
        contains relevant and active turbine information.

        Returns:
            xarray.Dataset: The processed wind turbine data, including information such as turbine IDs, hub height,
                            diameter, rated power, and geospatial coordinates (latitude, longitude) in WGS84 format.
        """
        df = pd.read_csv(self.turbine_csv, encoding='ISO-8859-1')
        df = df[df['operationalStatus'] == 'operationalStatus_normalbetrieb']
        lons, lats = self._convert_coordinates(df['x'].values, df['y'].values)
        df['longitude'], df['latitude'] = lons, lats
        selected_columns = ['xtf_id', 'hubHeight', 'diameter', 'ratedPower', 'latitude', 'longitude']
        return df[selected_columns].to_xarray()


    def _convert_coordinates(self, xs, ys):
        """
        Converts a batch of coordinates from the Swiss coordinate system to the WGS84 coordinate system. This method
        is designed to efficiently handle multiple coordinate conversions at once, which is particularly useful for
        processing datasets with multiple entries.

        Args:
            xs (array-like): Array of X-coordinates (eastings) in the Swiss coordinate system.
            ys (array-like): Array of Y-coordinates (northings) in the Swiss coordinate system.

        Returns:
            tuple: Two numpy arrays containing the converted longitudes and latitudes in the WGS84 system.
        """
        # Add progress bar for coordinate conversion
        with tqdm(total=len(xs), desc="Converting Coordinates") as pbar:
            for i in range(0, len(xs), 1000):  # Process in batches of 1000
                lons, lats = self.transformer.transform(xs[i:i+1000], ys[i:i+1000])
                if i == 0:
                    all_lons, all_lats = lons, lats
                else:
                    all_lons = np.concatenate((all_lons, lons))
                    all_lats = np.concatenate((all_lats, lats))
                pbar.update(min(1000, len(xs) - i))  # Update progress bar
        return all_lons, all_lats
    
        lons, lats = self.transformer.transform(xs, ys)
        return lons, lats
    

    def _load_and_process_buildings(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT EGID, GKODE, GKODN, GKAT FROM building WHERE GKAT IN (1020, 1040)")
        rows = cursor.fetchall()
        conn.close()
        
        # Add progress bar for processing buildings
        with tqdm(total=len(rows), desc="Processing Buildings") as pbar:
            for row in rows:
                # Your existing processing logic here
                pbar.update(1)  # Update progress bar
        gkodes, gkodns = zip(*[(row[1], row[2]) for row in rows if row[1] != '' and row[2] != ''])
        lons, lats = self._convert_coordinates(np.array(gkodes, dtype=float), np.array(gkodns, dtype=float))
        converted_data = [(rows[i][0], lats[i], lons[i], rows[i][3]) for i in range(len(lats))]
        return pd.DataFrame(converted_data, columns=['EGID', 'Latitude', 'Longitude', 'GKAT']).to_xarray()


    def haversine_vectorized(self, lat1, lon1, lat2, lon2):
        """
        Calculates distances between pairs of latitude and longitude points using the Haversine formula.

        Args:
            lat1 (numpy.ndarray): Array of latitudes for the first set of points.
            lon1 (numpy.ndarray): Array of longitudes for the first set of points.
            lat2 (numpy.ndarray): Array of latitudes for the second set of points.
            lon2 (numpy.ndarray): Array of longitudes for the second set of points.

        Returns:
            numpy.ndarray: Array of distances between each pair of points, in kilometers.
        """
        # Convert latitude and longitude from degrees to radians
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6371 * c  # Multiply by Earth's radius to get kilometers
        return km      

    def find_close_buildings(self, max_distance_km=1):
        """
        Identifies buildings that are within a specified distance from any wind turbine.

        Args:
            max_distance_km (float): Maximum distance in kilometers to consider a building close to a turbine.

        Returns:
            numpy.ndarray: Coordinates of buildings close to wind turbines.
        """
        buildings_coords = np.array([self.buildings_data['Latitude'].values, self.buildings_data['Longitude'].values]).T
        turbines_coords = np.array([self.turbines_data['latitude'].values, self.turbines_data['longitude'].values]).T
        distances = self.haversine_vectorized(buildings_coords[:, np.newaxis, 0], buildings_coords[:, np.newaxis, 1],
                                              turbines_coords[np.newaxis, :, 0], turbines_coords[np.newaxis, :, 1])
        min_distances = distances.min(axis=1)
        close_buildings_mask = min_distances <= max_distance_km
        for i in tqdm(range(len(distances)), desc="Finding Close Buildings"):
            min_distance = distances[i].min()
            close_buildings_mask[i] = min_distance <= max_distance_km
            
        return buildings_coords[close_buildings_mask]

    def analyze_proximity(self, max_distance_km=1):
        """
        Analyzes proximity of each building to wind turbines and returns details for those within a specified distance.

        Args:
            max_distance_km (float): Maximum distance in kilometers to consider for proximity analysis.

        Returns:
            numpy.ndarray: Array of tuples containing latitude, longitude, turbine index, and distance to the nearest turbine.
        """
        buildings_coords = np.array([self.buildings_data['Latitude'].values, self.buildings_data['Longitude'].values]).T
        turbines_coords = np.array([self.turbines_data['latitude'].values, self.turbines_data['longitude'].values]).T

        # Vectorized calculation of distances
        distances = self.haversine_vectorized(
            buildings_coords[:, np.newaxis, 0],
            buildings_coords[:, np.newaxis, 1],
            turbines_coords[np.newaxis, :, 0],
            turbines_coords[np.newaxis, :, 1]
        )

        # Find turbines within the specified distance
        close_turbines_mask = distances <= max_distance_km

        # Create results using broadcasting and filtering
        building_indices, turbine_indices = np.where(close_turbines_mask)
        results = np.array([
            (buildings_coords[i, 0], buildings_coords[i, 1], j, distances[i, j])
            for i, j in zip(building_indices, turbine_indices)
        ], dtype=[('Latitude', float), ('Longitude', float), ('Turbine Index', int), ('Distance to Turbine (km)', float)])

        return results
    

    def create_wind_turbines_list(self):
        """
        Creates a list of wind turbines with details from the CSV file.

        Returns:
            list: A list of dictionaries, each containing details of a wind turbine.
        """
        # Read the CSV file
        df = pd.read_csv(self.turbine_csv, encoding='ISO-8859-1')
        df = df[df['operationalStatus'] == 'operationalStatus_normalbetrieb']
        lons, lats = self._convert_coordinates(df['x'].values, df['y'].values)
        df['longitude'], df['latitude'] = lons, lats
        selected_columns = ['xtf_id', 'hubHeight', 'diameter', 'ratedPower', 'latitude', 'longitude']

        # Add progress bar for creating wind turbine list
        wind_turbines = []
        for _, row in tqdm(df[selected_columns].iterrows(), total=df[selected_columns].shape[0], 
                           desc="Creating Wind Turbines List"):
            turbine_dict = {
                "name": row['xtf_id'],  # Use the official turbine name
                "power": row['ratedPower'],
                "diameter": row['diameter'],
                "hub height": row['hubHeight'],
                "position": (row['latitude'], row['longitude'])
            }
            wind_turbines.append(turbine_dict)

        return wind_turbines


    
    def create_listening_points(self, data):
        # Add progress bar for creating listening points
        listening_points = []
        for x, b in tqdm(enumerate(data.tolist()), total=len(data), desc="Creating Listening Points"):
            point_dict = {"name": f"Listener {x+1}", "position": tuple(b)}
            listening_points.append(point_dict)

        return listening_points

In [14]:
# Usage
analysis = CASESTUDY1('Windraeder.csv', 'data.sqlite')
close_buildings = analysis.find_close_buildings() #find households within 1km distance to wind turbines
proximity_data = analysis.analyze_proximity() #calculate distance between each pair of houses in find_close_buildings and wind turbines in 
wind_turbines= analysis.create_wind_turbines_list()
listening_points=analysis.create_listening_points(close_buildings)

Creating Listening Points: 100%|██████████| 679/679 [00:00<00:00, 980018.04it/s]


In [25]:
listening_points[:10]

[{'name': 'Listener 1',
  'position': (47.1840355779907, 7.035464495295362),
  'L_den': 40.299308111089466},
 {'name': 'Listener 2',
  'position': (47.18278860777988, 7.0298278243390975),
  'L_den': 43.6562734704391},
 {'name': 'Listener 3',
  'position': (47.182807990689085, 7.033066458539599),
  'L_den': 41.43781793707756},
 {'name': 'Listener 4',
  'position': (47.18244895356822, 7.031672363974764),
  'L_den': 42.20894693131052},
 {'name': 'Listener 5',
  'position': (47.19168636620207, 7.018903279553709),
  'L_den': 43.3122544626612},
 {'name': 'Listener 6',
  'position': (47.18385991146152, 7.030251738559475),
  'L_den': 43.932869442575004},
 {'name': 'Listener 7',
  'position': (47.18597054635137, 7.019454891595439),
  'L_den': 50.39495605092774},
 {'name': 'Listener 8',
  'position': (47.18045021236114, 7.02666392948797),
  'L_den': 44.805583373122815},
 {'name': 'Listener 9',
  'position': (47.18540735936172, 7.018692014231644),
  'L_den': 49.69229247117846},
 {'name': 'Listene

In [15]:
wind_turbines

[{'name': 'turbine_5',
  'power': 2000,
  'diameter': 71,
  'hub height': 100,
  'position': (46.15984264103888, 7.037330571130646)},
 {'name': 'turbine_8',
  'power': 900,
  'diameter': 52,
  'hub height': 61,
  'position': (46.99000301237469, 8.086714691318104)},
 {'name': 'turbine_9',
  'power': 600,
  'diameter': 40,
  'hub height': 46,
  'position': (46.654223678580536, 8.616331670805865)},
 {'name': 'turbine_31',
  'power': 2000,
  'diameter': 82,
  'hub height': 78,
  'position': (47.30271124625021, 7.100814596378082)},
 {'name': 'turbine_32',
  'power': 2000,
  'diameter': 82,
  'hub height': 99,
  'position': (46.126610211902474, 7.0521936501789835)},
 {'name': 'turbine_33',
  'power': 900,
  'diameter': 44,
  'hub height': 55,
  'position': (46.654893308376046, 8.612465816047862)},
 {'name': 'turbine_34',
  'power': 2000,
  'diameter': 90,
  'hub height': 95,
  'position': (47.17689111885463, 7.0157053742490065)},
 {'name': 'turbine_35',
  'power': 2300,
  'diameter': 82,
  '

In [16]:
def count_buildings(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Query to count all buildings
    query_all = "SELECT COUNT(*) FROM building"
    cursor.execute(query_all)
    total_buildings = cursor.fetchone()[0]

    # Query to count buildings in categories 1020 and 1040
    query_specific = "SELECT COUNT(*) FROM building WHERE GKAT IN (1020, 1040)"
    cursor.execute(query_specific)
    specific_buildings = cursor.fetchone()[0]

    # Close the database connection
    conn.close()

    return total_buildings, specific_buildings

In [17]:
count_buildings('data.sqlite')

(3210676, 1673493)

In [18]:
from windwhisper import windturbines

In [19]:
wt = windturbines.WindTurbines(wind_turbines=wind_turbines, listeners=listening_points,)

In [20]:
wt.fetch_noise_map()

In [21]:
wt.fetch_wind_speeds()

Starting concurrent data download for all turbines...
Retrying download for turbine_5 (Attempt 1/10)
Done.


/Users/maximebalandret/opt/anaconda3/envs/noise/lib/python3.10/site-packages/numpy/core/numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


In [22]:
wt.analyze_noise()

In [23]:
wt.na.analyze_and_calculate_lden2()

interpolate noise for every wind speeds at wind turbine!
step 2/5 done!
noise emissions separated as day/evening/night emissions
Lden calculated!
wt.listeners updated with Lden!


({'Listener 1': <xarray.DataArray ()>
  array(40.29930811)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 2': <xarray.DataArray ()>
  array(43.65627347)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 3': <xarray.DataArray ()>
  array(41.43781794)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 4': <xarray.DataArray ()>
  array(42.20894693)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 5': <xarray.DataArray ()>
  array(43.31225446)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 6': <xarray.DataArray ()>
  array(43.93286944)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 7': <xarray.DataArray ()>
  array(50.39495605)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 8': <xarray.DataArray ()>
  array(44.80558337)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 9': <xarray.DataArray ()>
  array(49.69229247)
  Coordinates:
      turbine  <U10 'turbine_91',
  'Listener 10': <xarray.Dat

In [24]:
wt.na.display_listeners_on_map_with_Lden()

In [29]:
wt.noise_map.get_altitudes_between_points('Listener 107','turbine_59')

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U1'), dtype('float64')) -> None

In [11]:
turbine_names = []
for turbine in wt.wind_turbines:
    turbine_names.append(turbine['name'])

closest_turbine_distance = {}

# Loop over each listener
for listener in wt.listeners:
    listener_name = listener['name']  # Extract the name of the current listener.
    # Find the distance to each turbine and get the minimum
    min_distance = min(d[(turbine['name'], listener_name)]['distance'] for turbine in wt.wind_turbines)
    closest_turbine_distance[listener_name] = min_distance

NameError: name 'd' is not defined

In [8]:
interpolation=wt.na.interpolate_noise3(wt.ws.wind_speed,wt.noise_map.individual_noise)

In [9]:
cumulative_dB=wt.na.calculate_cumulative_dB(interpolation)

In [10]:
noise_separated=wt.na.separate_noise_emissions(cumulative_dB)

In [11]:
Lden=wt.na.compute_lden(noise_separated)

In [12]:
wt.na.update_listeners_with_lden(Lden)

[{'name': 'Listener 1',
  'position': (47.1840355779907, 7.035464495295362),
  'L_den': 33.73630787702028},
 {'name': 'Listener 2',
  'position': (47.18278860777988, 7.0298278243390975),
  'L_den': 35.98592001689204},
 {'name': 'Listener 3',
  'position': (47.182807990689085, 7.033066458539599),
  'L_den': 34.50310064222377},
 {'name': 'Listener 4',
  'position': (47.18244895356822, 7.031672363974764),
  'L_den': 35.02831614462003},
 {'name': 'Listener 5',
  'position': (47.19168636620207, 7.018903279553709),
  'L_den': 35.68537880645304},
 {'name': 'Listener 6',
  'position': (47.18385991146152, 7.030251738559475),
  'L_den': 36.12126352891481},
 {'name': 'Listener 7',
  'position': (47.18597054635137, 7.019454891595439),
  'L_den': 40.406196445442085},
 {'name': 'Listener 8',
  'position': (47.18045021236114, 7.02666392948797),
  'L_den': 36.911734753352725},
 {'name': 'Listener 9',
  'position': (47.18540735936172, 7.018692014231644),
  'L_den': 40.02155810695061},
 {'name': 'Listen

In [8]:
Lden=wt.na.analyze_and_calculate_lden()

<xarray.DataArray 'WS10' (turbine: 41, time: 35088)>
array([[ 5.0573683,  4.752873 ,  4.9840465, ...,  3.020379 ,  2.7387478,
         2.9565039],
       [ 4.379362 ,  4.2954345,  4.3639555, ...,  8.859892 ,  9.296843 ,
         9.0928545],
       [ 2.7756667,  3.221314 ,  3.4752238, ..., 12.096262 , 11.478349 ,
        10.675275 ],
       ...,
       [ 9.801038 , 10.031105 , 10.18939  , ...,  2.311985 ,  2.0408485,
         1.8614213],
       [ 9.801038 , 10.031105 , 10.18939  , ...,  2.311985 ,  2.0408485,
         1.8614213],
       [ 9.801038 , 10.031105 , 10.18939  , ...,  2.311985 ,  2.0408485,
         1.8614213]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01 ... 2017-12-31T23:30:00
  * turbine  (turbine) <U10 'Turbine 1' 'Turbine 2' ... 'Turbine 50'
Attributes:
    grid_mapping:   crs
    long_name:      Wind Speed at 10m
    standard_name:  wind_speed
    units:          m s-1
['Turbine 1' 'Turbine 2' 'Turbine 3' 'Turbine 9' 'Turbine 10' 'Turbine 1

Calculating Lden values: 100%|██████████| 679/679 [43:22<00:00,  3.83s/it]  


In [12]:
Lden

{'Listener 673': 60.67838504003782,
 'Listener 565': 52.02396068130096,
 'Listener 146': 60.076969858752065,
 'Listener 654': 60.67838504003782,
 'Listener 306': 56.66690688389892,
 'Listener 366': 52.02396068130096,
 'Listener 351': 56.66690688389892,
 'Listener 418': 56.66690688389892,
 'Listener 49': 61.312097275138285,
 'Listener 28': 56.059932244139475,
 'Listener 624': 60.67838504003782,
 'Listener 68': 61.47036156993879,
 'Listener 214': 56.66690688389892,
 'Listener 386': 56.66690688389892,
 'Listener 460': 56.63376287404467,
 'Listener 538': 56.63376287404467,
 'Listener 480': 56.63376287404467,
 'Listener 301': 56.66690688389892,
 'Listener 508': 56.63376287404467,
 'Listener 369': 52.02396068130096,
 'Listener 422': 56.66690688389892,
 'Listener 19': 56.059932244139475,
 'Listener 184': 60.076969858752065,
 'Listener 533': 56.63376287404467,
 'Listener 618': 60.67838504003782,
 'Listener 519': 56.63376287404467,
 'Listener 411': 56.66690688389892,
 'Listener 425': 56.6669068

In [35]:
wt.listeners

[{'name': 'Listener 1',
  'position': (47.1840355779907, 7.035464495295362),
  'Lden_value': 58.29889851479544},
 {'name': 'Listener 2',
  'position': (47.18278860777988, 7.0298278243390975),
  'Lden_value': 58.395118344993136},
 {'name': 'Listener 3',
  'position': (47.182807990689085, 7.033066458539599),
  'Lden_value': 58.72963959503079},
 {'name': 'Listener 4',
  'position': (47.18244895356822, 7.031672363974764),
  'Lden_value': 58.395118344993136},
 {'name': 'Listener 5',
  'position': (47.19168636620207, 7.018903279553709),
  'Lden_value': 58.395118344993136},
 {'name': 'Listener 6',
  'position': (47.18385991146152, 7.030251738559475),
  'Lden_value': 58.395118344993136},
 {'name': 'Listener 7',
  'position': (47.18597054635137, 7.019454891595439),
  'Lden_value': 58.395118344993136},
 {'name': 'Listener 8',
  'position': (47.18045021236114, 7.02666392948797),
  'Lden_value': 58.395118344993136},
 {'name': 'Listener 9',
  'position': (47.18540735936172, 7.018692014231644),
  'L

In [30]:
wt.wind_turbines

[{'name': 'Turbine 1',
  'power': 2000.0,
  'diameter': 71.0,
  'hub height': 100.0,
  'position': (46.15984264103888, 7.037330571130646)},
 {'name': 'Turbine 2',
  'power': 900.0,
  'diameter': 52.0,
  'hub height': 61.0,
  'position': (46.99000301237469, 8.086714691318104)},
 {'name': 'Turbine 3',
  'power': 600.0,
  'diameter': 40.0,
  'hub height': 46.0,
  'position': (46.654223678580536, 8.616331670805865)},
 {'name': 'Turbine 9',
  'power': 2000.0,
  'diameter': 82.0,
  'hub height': 78.0,
  'position': (47.30271124625021, 7.100814596378082)},
 {'name': 'Turbine 10',
  'power': 2000.0,
  'diameter': 82.0,
  'hub height': 99.0,
  'position': (46.126610211902474, 7.0521936501789835)},
 {'name': 'Turbine 11',
  'power': 900.0,
  'diameter': 44.0,
  'hub height': 55.0,
  'position': (46.654893308376046, 8.612465816047862)},
 {'name': 'Turbine 12',
  'power': 2000.0,
  'diameter': 90.0,
  'hub height': 95.0,
  'position': (47.17689111885463, 7.0157053742490065)},
 {'name': 'Turbine 13

In [13]:
for listener in wt.listeners:
    listener_name = listener['name']
    if listener_name in Lden:
        listener['Lden_value'] = Lden[listener_name]
    else:
        listener['Lden_value'] = None  # or any default value you prefer

In [11]:
import folium 
# Get the average latitude and longitude to center the map
avg_lat = sum(listener['position'][0] for listener in wt.listeners) / len(wt.listeners)
avg_lon = sum(listener['position'][1] for listener in wt.listeners) / len(wt.listeners)

# Create a folium map centered at the average latitude and longitude
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=7)

# Add markers for each listener with their respective Lden value
for listener in wt.listeners:
    folium.Marker(
        location=listener['position'],
        tooltip=f"{listener['name']}: Lden {listener['Lden']} dB",
        icon=folium.Icon(icon="info-sign", color="green")
    ).add_to(m)

for turbine in wt.wind_turbines:
    folium.Marker(
        location=turbine['position'],
        tooltip=f"{turbine['name']}",
        icon=folium.Icon(icon="info-sign", color="red")).add_to(m)
    
# Display the map
display(m)

KeyError: 'Lden'